## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-20-51-22 +0000,startribune,"Live: Food, fun, crowds on first day of the St...",https://www.startribune.com/mn-state-fair-food...
1,2025-08-21-20-47-44 +0000,startribune,Live: DFL is revoking Omar Fateh’s endorsement...,https://www.startribune.com/minneapolis-mayor-...
2,2025-08-21-20-46-18 +0000,nyt,Hurricane Erin Is Heading Out to Sea. People A...,https://www.nytimes.com/2025/08/21/us/hurrican...
3,2025-08-21-20-41-53 +0000,bbc,"Israel will begin talks to free all hostages, ...",https://www.bbc.com/news/articles/c754kknw2g2o...
4,2025-08-21-20-39-00 +0000,wsj,A Georgia-based reporter has sued the Trump ad...,https://www.wsj.com/politics/trump-immigration...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,56
105,new,21
443,ukraine,16
449,russia,14
21,israel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2025-08-21-20-25-04 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...,125
35,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,121
146,2025-08-21-15-01-46 +0000,nypost,Trump’s massive $500M civil fraud fine in AG T...,https://nypost.com/2025/08/21/us-news/trumps-m...,118
7,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,118
328,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,125,2025-08-21-20-25-04 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
97,66,2025-08-21-17-07-24 +0000,nypost,North Korea’s top officers abruptly pulled out...,https://nypost.com/2025/08/21/world-news/north...
229,52,2025-08-21-09-39-40 +0000,bbc,Palestinians flee Gaza City districts as Israe...,https://www.bbc.com/news/articles/clyr7l0z9edo...
192,51,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
129,48,2025-08-21-15-31-16 +0000,startribune,New women-led clinic network in Twin Cities ho...,https://www.startribune.com/women-led-clinics-...
292,46,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...
133,45,2025-08-21-15-24-02 +0000,nypost,Husband of missing California mom of four char...,https://nypost.com/2025/08/21/us-news/nikki-sa...
4,43,2025-08-21-20-39-00 +0000,wsj,A Georgia-based reporter has sued the Trump ad...,https://www.wsj.com/politics/trump-immigration...
255,39,2025-08-21-05-56-59 +0000,nypost,Screaming DC man who tried to run from federal...,https://nypost.com/2025/08/21/us-news/dc-migra...
80,37,2025-08-21-17-41-08 +0000,bbc,"Record 111,000 UK asylum applications in past ...",https://www.bbc.com/news/articles/cwy1kxv8xewo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
